# Harvesting data from the web: APIs  

### A first API

[Chronicling America](http://chroniclingamerica.loc.gov/about/) is a joint project of the National Endowment for the Humanities and the Library of Congress .

Search for articles that mention "[slavery](http://chroniclingamerica.loc.gov/search/pages/results/?andtext=slavery)".

![](images/chron.png)

![](images/chron_slavery.png)

<div class="alert alert-info">

Look at the URL. What happens if you change the word slavery to abolition? 

What happens to the URL when you go to the second page? Can you get to page 251?

</div>

What if we append ``&format=json`` to the end of the search URL? 


http://chroniclingamerica.loc.gov/search/pages/results/?andtext=slavery&format=json


![](images/chron_json.png)

[``requests``](http://docs.python-requests.org/en/master/) is a useful and commonly used HTTP library for python. It is not a part of the default installation, but is included with Anaconda Python Distribution. 

In [ ]:
import requests

It would be possible to use the API URL and parameters directly in the requests command, but since the most likely scenario involves making repeating calls to ``requests`` as part of a loop -- the search returned less than 1% of the results -- I store the strings first. 

In [ ]:
base_url   = 'http://chroniclingamerica.loc.gov/search/pages/results/'
parameters = '?andtext=slavery&format=json'

`requests.get()` is used for both accessing websites and APIs. The command can be modified by several arguements, but at a minimum, it requires the URL.

In [ ]:
r = requests.get(base_url + parameters)

`r` is a `requests` response object. Any JSON returned by the server are stored in `.json().`

In [ ]:
search_json = r.json()

JSONs are dictionary like objects, in that they have keys (think variable names) and values. `.keys()` returns a list of the keys.

In [ ]:
search_json.keys()

You can return the value of any key by putting the key name in brackets.

In [ ]:
search_json['totalItems']

<div class="alert alert-info">
What else is in there? Where is the stuff we want?
</div>

As is often the case with results from an API, most of the keys and values are metadate about either the search or what is being returned. These are useful for knowing if the search is returning what you want, which is particularly important when you are making multiple calls to the API. 

The data I'm intereted in is all in `items`. 

In [ ]:
type(search_json['items'])

In [ ]:
len(search_json['items'])

`items` is a list with 20 items.

In [ ]:
type(search_json['items'][3])

Each of the 20 items in the list is a dictionary. 

In [ ]:
first_item = search_json['items'][0]

first_item.keys()

<div class="alert alert-info">
What is the title of the first item?
</div>

While a standard CSV file has a header row that describes the contents of each column, a JSON file has keys identifying the values found in each case. Importantly, these keys need not be the same for each item. Additionally, values don't have to be numbers of strings, but could be lists or dictionaries. For example, this JSON could have included a `newspaper` key that was a dictionary with all the metadata about the newspaper the article and issue was published, an `article` key that include the article specific information as another dictionary, and a `text` key whose value was a string with the article text.

As before, we can examine the contents of a particular item, such as the publication's `title`.

In [ ]:
first_item['ocr_eng']

In [ ]:
print(first_item['ocr_eng'])

In [ ]:
print(first_item['ocr_eng'][:200])

The easiest way to view or analyze this data is to convert it to a dataset-like structure. While Python does not have a builting in dataframe type, the popular `pandas` library does. By convention, it is imported as `pd`.

In [ ]:
import pandas as pd

# Make sure all columns are displayed
pd.set_option("display.max_columns",101)

pandas is prety smart about importing different JSON-type objects and converting them to dataframes with its `.DataFrame()` function.

In [ ]:
df = pd.DataFrame(search_json['items'])

df.head(6)

Note that I converted `search_json['items']` to  dataframe and not the entire JSON file. This is because I wanted each row to be an article. 

In [ ]:
pd.DataFrame(search_json)

If this dataframe contained all the items that you were looking for, it would be easy to save this to a csv file for storage and later analysis.

In [ ]:
df.to_csv('lynching_articles.csv')

In [ ]:
df.to_csv('lynching_articles.csv', encoding='utf8')

In [ ]:
!head lynching_articles.csv

<div class="alert alert-info">
<h3> Your turn</h3>
<p> Conduct your own search of the API. Store the results in a csv file.

</div>



![](images/exchange.png)

In [ ]:
r = requests.get('https://api.exchangeratesapi.io/latest?base=USD')

In [ ]:
pd.DataFrame(r.json())

<div class="alert alert-info">
<h3> Your turn</h3>
<p> Conduct your own search of the API. Change the base rate to Euros. Store the results in a csv file.

</div>




In [ ]:
'https://api.exchangeratesapi.io/latest?base=USD'

In [ ]:
base_url = 'https://api.exchangeratesapi.io/latest'

In [ ]:
parameters = {'base': 'USD'}

In [ ]:
r = requests.get(base_url, 
                 params = parameters)

In [ ]:
pd.DataFrame(r.json())

<div class="alert alert-info">
<h3> Your turn</h3>
<p>What is the current exchange rate using the Japanese yen (JPY) as the base rate? Use it as a parameter.
</div>

### Bringing functions back in.

In [ ]:
base_url = 'https://api.exchangeratesapi.io/latest'
parameters = {'base': 'USD'}
r = requests.get(base_url, params = parameters)
df = pd.DataFrame(r.json())

Spot the difference...

In [ ]:
currency = 'USD'
base_url = 'https://api.exchangeratesapi.io/latest'
parameters = {'base': currency}
r = requests.get(base_url, params = parameters)
df = pd.DataFrame(r.json())

![](images/function.png)

In [ ]:
def get_exchange(currency):
    base_url = 'https://api.exchangeratesapi.io/latest'
    parameters = {'base': currency}
    r = requests.get(base_url, params = parameters)
    df = pd.DataFrame(r.json())
    return df

In [ ]:
get_exchange('USD')

<div class="alert alert-info">
<h3> Your turn</h3>
<p>What is the current exchange rate using the Russina ruble (RUB) as the base rate? Use the function and save the results to a csv file.
</div>

This is only a small subset of the articles on lynching that are available, however. The API returns results in batches of 20 and this is only the first page of results. As is often the case, I'll need to make multiple calls to the API to retrieve all the data of interest. The easiest way to do that is to define a small function for getting the article information and put that in a loop. While it isn't a requirement that you create a function for making the API call, it will make your code easier to read and debug.


Looking at the API guidelines, there is an additional paramater `page` that tells the API which subset of results we want. This name varies by API but their is usually some mechanism for retrieiving results beyond the initial JSON.

Before creating the loop and making multiple calls to the API, I want to make sure that the API is working the way I think it is. 

<div class="alert alert-info">
Back to the newspapers. Look at the API guidelines. How can we get the third page?
</div>


[Guidelines](https://chroniclingamerica.loc.gov/about/api/)

In [ ]:
base_url   = 'http://chroniclingamerica.loc.gov/search/pages/results/'
parameters = '?andtext=slavery&format=json&page=3'

r = requests.get(base_url + parameters)
results =  r.json()


In [ ]:

print(results['startIndex'])
print(results['endIndex'])

A call to random selected page 3 returns results 41 through 60, which is what I expected since each page has 20 items.

The parameters are getting pretty ugly, so fortunately `requests` accepts a dictionary where the keys are the parameter names as defined by the API and the values are the search paramaters you are looking for. So the same request can be rewritten:

In [ ]:
base_url = 'http://chroniclingamerica.loc.gov/search/pages/results/'
parameters = {'andtext': 'lynching',
              'page'   : 3,
              'format'  : 'json'}

r = requests.get(base_url, params=parameters)

results =  r.json()

print(results['startIndex'], results['endIndex'])

This can be rewritten as function:

In [ ]:
def get_articles():
    '''
    Make calls to the Chronicling America API.
    '''
    
    base_url   = 'http://chroniclingamerica.loc.gov/search/pages/results/'
    parameters = {'andtext': 'lynching',
                  'page'   : 3,
                  'format' : 'json'}
    
    r = requests.get(base_url, params = parameters)
    results =  r.json()
    
    return results

In [ ]:
results = get_articles()

results['startIndex'], results['endIndex']

The advantage of writing a function, however, would be that you can pass along your own parameters, such as the search term and page number, which would make this much more useful. 

In [ ]:
def get_articles(search_term, page_number):
    '''
    Make calls to the Chronicling America API.
    '''
    
    base_url = 'http://chroniclingamerica.loc.gov/search/pages/results/'
    parameters = {'andtext': search_term,
                  'page'   : page_number,
                  'format' : 'json'}
    
    r = requests.get(base_url, params = parameters)
    results =  r.json()
    new_df = pd.DataFrame(results['items'])

    return new_df

In [ ]:
results = get_articles('lynching', 2)

results.head()

In [ ]:
results = get_articles('cows', 45)

results.head()

Back to Chronicling America. Now, the first 60 results could downloaded in a just a few lines:

In [ ]:
for page_number in [1, 2, 3]: 
    print(page_number)
    

In [ ]:
for page_number in range(1, 4): 
    print(page_number)
    

In [ ]:
for page_number in range(1,4):
    
    results = get_articles('lynching', page_number)
    

In [ ]:
len(results)

Everything appears to be working, but unfortunately I only have the last page of results still. Each call to the API was redefining `results` variable. In this case, I set up an empty dataframe to store the results and will append the items from each page of results.

In [ ]:
dfs = [] # empty list to store dataframes

for page_number in range(1,4):
    new_df = get_articles('lynching', page_number)
    dfs.append(new_df) 

df = pd.concat(dfs, ignore_index = True)
df.info()

For a large download, you would still want to tweak this a bit by pausing between each API call and making it robust to internet or API errors, but this is a solid framework for collecting data from an API.

In [ ]:
from time import sleep

In [ ]:
dfs = [] # empty list to store dataframes

for page_number in range(1,4):
    new_df = get_articles('lynching', page_number)    
    dfs.append(new_df) 
    sleep(1)
    print('Getting page:',page_number)
    
df = pd.concat(dfs, ignore_index = True)
df.info()

### How about Twitter?

In [ ]:
from twython import Twython

Sign up as a developer

In [ ]:
APP_KEY            = 'J8TGgv1SlKgAtqvxGZzc9XiNx'
APP_SECRET         = '8bEieGM73FLqbnWu6WcTR3vM6ICfEBEmQ8lXgqojw5IL1uzQ0Z'
OAUTH_TOKEN        = '594565064-erQRFPOFk520ePJjR86b9H2PTPxAF9i1d3A7pzjp'
OAUTH_TOKEN_SECRET = 'qGwK0Jz7f0YyYlQIoLSJ8FJCIX3ydFQBo0yJOePsaWXL9'



Store your credentials

In [ ]:
twitter = Twython(APP_KEY,
                  APP_SECRET,
                  OAUTH_TOKEN,
                  OAUTH_TOKEN_SECRET)

Start your searches!

In [ ]:
user_timeline = twitter.get_user_timeline(screen_name='oprah')

In [ ]:
pd.DataFrame(user_timeline)



<div class="alert alert-info">
<h3> Your turn</h3>
<p> Find the tweets from someone else. If you add <code>, count = 200 </code> after the username, you can get up to 200 tweets. Do it!

</div>



In [ ]:
python_tweets = twitter.search(q='ipynb', count=200)

In [ ]:
pd.DataFrame(python_tweets['statuses'])

In [ ]:
python_tweets

In [ ]:
python_tweets['search_metadata']

In [ ]:
python_tweets.keys()

In [ ]:
pd.DataFrame(python_tweets['statuses'])

In [ ]:
python_tweets = twitter.search(q     = 'ipynb', 
                               count = 200,
                               maxid = 1023186440514142207)                        

In [ ]:
pd.DataFrame(python_tweets['statuses'])

In [ ]:
df = pd.DataFrame(python_tweets['statuses'])
df['text'].values

In [ ]:
for status_update in df['text'].values:
    print(status_update)



<div class="alert alert-info">
<h3> Your turn</h3>
<p> Do a search!
</div>




Get 3,200 tweets from somone

In [ ]:
pd.DataFrame(user_timeline).iloc[-1]['id']

In [ ]:
user_timeline = twitter.get_user_timeline(screen_name='oprah', 
                                         count = 200,
                                         max_id = 929540230465458177)

In [ ]:
def get_timeline(screen_name):
    tweets = []
    user_timeline = twitter.get_user_timeline(screen_name=screen_name,
                                              count = 200)
    df = pd.DataFrame(user_timeline)
    tweets.append(df)
    
    most_recent = pd.DataFrame(user_timeline).iloc[-1]['id']
    
    for i in range(0,15):
        user_timeline = twitter.get_user_timeline(screen_name=screen_name,
                                              count = 200,
                                                 max_id = most_recent)
        df = pd.DataFrame(user_timeline)
        tweets.append(df)
        most_recent = pd.DataFrame(user_timeline).iloc[-1]['id']
    
    tweet_df = pd.concat(tweets, ignore_index = True)
    return tweets    
    